In [1]:
import pandas as pd
import pandas_gbq as pdq
from google.cloud import bigquery
client = bigquery.Client(project='analytics-supplychain-thd')
import pulp
import numpy as np
import cplex
# import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
## Getting regions and initializing dictionaries

# regions_query = """
#                 SELECT DISTINCT REGION
#                 FROM `analytics-supplychain-thd.SRS_Matching.DEMAND_DATA`
# """
# regions_data = pdq.read_gbq(regions_query, project_id= 'analytics-supplychain-thd' , dialect='standard')
# regions = regions_data['REGION'].tolist()

ded_details_dict = {}
ow_dict = {}
summary_dict = {}
print('Created results dictionaries')

Created results dictionaries


In [4]:
# for region in regions:
description = "SRS Matching"

start_time = datetime.strptime(datetime.now().strftime("%H:%M:%S"),"%H:%M:%S")   

############## INPUT QUERIES #########
q1 = """
        SELECT DISTINCT 
            --THD_SRS,
            CAST(PAIR_NUMBER AS STRING) AS pair_number
            ,loads
            ,OW_COST_PER_LOAD AS OW_CPL
        FROM `analytics-supplychain-thd.SRS_Matching.DEMAND_DATA`
"""
sqllanes = pdq.read_gbq(q1, project_id= 'analytics-supplychain-thd' , dialect='standard')

q2 = """
        SELECT ROUTE_NBR
            ,HOME_DC
            ,CAST(CASE WHEN A_LOADED_IND = 1 THEN PAIR_NBR1 ELSE null END AS STRING) AS lane_1
            ,CAST(CASE WHEN B_LOADED_IND = 1 THEN PAIR_NBR2 ELSE null END AS STRING) AS lane_2
            ,CAST(CASE WHEN C_LOADED_IND = 1 THEN PAIR_NBR3 ELSE null END AS STRING) AS lane_3
            ,CAST(CASE WHEN D_LOADED_IND = 1 THEN PAIR_NBR4 ELSE null END AS STRING) AS lane_4
            ,TOTAL_DIST AS total_dist
            ,RF_COST AS cost
            ,mode
        FROM `analytics-supplychain-thd.SRS_Matching.rf_routes_final`
"""
sqlroutedata = pdq.read_gbq(q2, project_id= 'analytics-supplychain-thd' , dialect='standard')

q3 = """
        SELECT DISTINCT 
            CAST(PAIR_NUMBER AS STRING) AS pair_number
        FROM `analytics-supplychain-thd.SRS_Matching.DEMAND_DATA`
        WHERE O_TYPE IN ('BDC', 'FDC', 'FDC/BDC', 'BDC/FDC') AND D_TYPE IN ('STR')
"""
sqldcstr = pdq.read_gbq(q3, project_id= 'analytics-supplychain-thd' , dialect='standard')

q4 = """
        SELECT DISTINCT 
            CAST(PAIR_NUMBER AS STRING) AS pair_number
        FROM `analytics-supplychain-thd.SRS_Matching.DEMAND_DATA`
        WHERE LEG LIKE 'SRS%'
"""
sqlsrs = pdq.read_gbq(q4, 
                      project_id= 'analytics-supplychain-thd' , 
                      dialect='standard')


a = datetime.strptime(datetime.now().strftime("%H:%M:%S"),"%H:%M:%S")
print(a - start_time, " data pull from BQ")

routenbr = {}
for ind, row in sqlroutedata.iterrows():
    route_nbr = row['ROUTE_NBR']
    routenbr[(route_nbr)] = []

lanes = {}
for ind, row in sqllanes.iterrows():
    pair_nbr = row['pair_number']
    lanes[(pair_nbr)] = []

dc_str_lanes = []
for ind,row in sqldcstr.iterrows():
    pair_nbr = row['pair_number']
    dc_str_lanes.append(pair_nbr)

srs_lanes = {}
for ind,row in sqlsrs.iterrows():
    pair_nbr = row['pair_number']
    srs_lanes[(pair_nbr)] = []

routecost = {}
routepairs = {}
routedc = {}
routedists = {}
for ind,row in sqlroutedata.iterrows():
    route_nbr, route_cost, route_dc, route_dist, lane_1, lane_2, lane_3, lane_4 =\
    row['ROUTE_NBR'], row['cost'], row['HOME_DC'], row['total_dist'], row['lane_1'], row['lane_2'], row['lane_3'], row['lane_4']

    routecost[(route_nbr)] = route_cost
    routepairs[(route_nbr)] = (lane_1, lane_2, lane_3, lane_4)
    routedc [(route_nbr)] = route_dc
    routedists[(route_nbr)] = route_dist

owcost = {}
demand = {}
for ind,row in sqllanes.iterrows():
    pair_nbr, loads, ow_cost = row['pair_number'], row['loads'], row['OW_CPL']
    owcost[(pair_nbr)] = ow_cost
    demand[(pair_nbr)] = loads

b = datetime.strptime(datetime.now().strftime("%H:%M:%S"),"%H:%M:%S")
print(b - a, " data processing, creating dictionaries")

########## MODEL DEFN #############
prob = pulp.LpProblem("SRS Matching", pulp.LpMinimize)

###### DVARS ####
ded = pulp.LpVariable.dicts("Dedicated", ((route_nbr) for route_nbr in routenbr.keys()), lowBound=0, cat = pulp.LpInteger)
ow = pulp.LpVariable.dicts("One_Way", ((pair_nbr) for pair_nbr in lanes.keys()), lowBound=0, cat=pulp.LpInteger)
thd_cover = pulp.LpVariable.dicts("THD Cover", ((pair_nbr) for pair_nbr in srs_lanes.keys()), lowBound=0, cat=pulp.LpInteger)

lane_ded = pulp.LpVariable.dicts("Lane Ded", ((lane) for lane in lanes.keys()), cat = pulp.LpBinary)
lane_ow = pulp.LpVariable.dicts("Lane OW", ((lane) for lane in lanes.keys()), cat = pulp.LpBinary)

a = datetime.strptime(datetime.now().strftime("%H:%M:%S"),"%H:%M:%S")
print(a - b, " model set up")

########## OBJ FUNCTION ##########
prob += pulp.lpSum([ded[(route_nbr)]*routecost[(route_nbr)] 
                    for route_nbr in routenbr.keys()]) - \
        pulp.lpSum([thd_cover[(pair_number)]*owcost[(pair_number)] 
                    for pair_number in srs_lanes.keys()])

########### CONSTRAINTS #######
###Demand
for pair_nbr in demand.keys():
    routes_with_lane = {k:v for (k,v) in routepairs.items() if pair_nbr in v}
    prob += (ow[pair_nbr] + pulp.lpSum([ded[(rn)] for rn in routes_with_lane.keys()]) == demand[(pair_nbr)]
                ,"Demand Constraint for lane number: {}".format(pair_nbr))
    
for lane in lanes.keys():
    routes_with_lane = {k:v for (k,v) in routepairs.items() if lane in v}
    prob += pulp.lpSum([ded[(rn)] for rn in routes_with_lane.keys()]) <= 100000*lane_ded[(lane)]
    prob += ow[(lane)] <= 100000*lane_ow[(lane)]

for lane in lanes.keys():
    prob += lane_ded[(lane)] + lane_ow[(lane)] >= 1

for dc_str_lane in dc_str_lanes:
    prob += lane_ow[(dc_str_lane)] == 0

for srs_lane in list(srs_lanes.keys()):
    routes_with_srs_lane = {k:v for (k,v) in routepairs.items() if srs_lane in v} 
    prob += thd_cover[(srs_lane)] == pulp.lpSum([ded[(rn)] for rn in routes_with_srs_lane.keys()])

b = datetime.strptime(datetime.now().strftime("%H:%M:%S"),"%H:%M:%S")
print(b - a, " constraints set up")

######### SOLVING THE MODEL #############
solver_cplex = pulp.CPLEX_PY(timeLimit = 1800, gapRel = 0.01)
prob.solve(solver_cplex)

a = datetime.strptime(datetime.now().strftime("%H:%M:%S"),"%H:%M:%S")
print(a - b, " model solved ", pulp.LpStatus[prob.status])

solution = pd.DataFrame(columns=["var", "val"])
for v in prob.variables():
    if v.varValue != 0:
        new_row = pd.DataFrame({"var": [v.name], "val": [v.varValue]})
        solution = pd.concat([solution, new_row], ignore_index=True)
        # solution = solution.append({"var": v.name, "val": v.varValue}, ignore_index=True)

one_way = solution[solution['var'].str.contains("One_Way")]
dedicated = solution[solution['var'].str.contains("Dedicated")]

one_way.columns = ['pair_number', 'LOADS']
dedicated.columns = ['ROUTE_NBR', 'LOADS']

one_way['pair_number'] = one_way['pair_number'].str.replace("One_Way_", "", case=False)
dedicated['ROUTE_NBR'] = dedicated['ROUTE_NBR'].str.replace("Dedicated_", "", case=False)

dedicated['DESCRIPTION'] = description

dedicated = dedicated[['ROUTE_NBR','LOADS','DESCRIPTION']]
dedicated = dedicated.astype({ 
                                'ROUTE_NBR': 'int',
                                'LOADS': 'int', 
                                'DESCRIPTION': 'str'})

# dedicated['RUN_DATE'] = datetime.now().date()

ded_details = pd.merge(dedicated, sqlroutedata[['ROUTE_NBR', 'HOME_DC', 'cost', 'total_dist', 'mode']] \
                        , on = ['ROUTE_NBR'], how = 'left')
ded_details.rename(columns={'cost': 'COST',
                            'total_dist': 'DIST'},
                    inplace=True)
ded_details['Run_Date'] = str(datetime.now().date())
ded_details['Run_Time'] = str(datetime.now().time().strftime("%H:%M"))
one_way['DESCRIPTION'] = description

one_way = one_way[['pair_number','LOADS', 'DESCRIPTION']]
one_way = one_way.astype({
                             
                            'pair_number': 'str',
                            'LOADS': 'int',
                            'DESCRIPTION': 'str'
                        })
one_way.rename(columns={'pair_number': 'PAIR_NUMBER'}, inplace=True)

ded_cost_df = pd.merge(dedicated, sqlroutedata[['ROUTE_NBR','cost']],\
            how = 'left', left_on='ROUTE_NBR', right_on='ROUTE_NBR')
ded_cost = sum(ded_cost_df['LOADS'] * ded_cost_df['cost'])

ow_cost_df = pd.merge(one_way, sqllanes,\
            how = 'left', left_on='PAIR_NUMBER', right_on='pair_number')
ow_cost = sum(ow_cost_df['LOADS'] * ow_cost_df['OW_CPL'])

summary = pd.DataFrame(columns=['region','dedicated_cost','ow_cost','total_cost','description'])
summary = pd.concat([summary, 
                        pd.DataFrame({
                        'dedicated_cost': [ded_cost], 
                        'ow_cost': [ow_cost],
                        'total_cost': [pulp.value(prob.objective)], 
                        'description': [description]})] 
                        ,ignore_index=True
                        )

# ow_dict[(region)] = one_way
# ded_details_dict[(region)] = ded_details
# summary_dict[(region)] = summary

end_time = datetime.strptime(datetime.now().strftime("%H:%M:%S"),"%H:%M:%S")
print(end_time- a, " results uploaded")

print("INPUT------ Demand Lanes:",len(demand),", Route Choices:",len(routenbr))
print("OUTPUT----- Routes Chosen:",dedicated.shape[0],", One Way Lanes:",one_way.shape[0])
print("Completed, Total time: ", end_time - start_time, ", Total cost:",\
        round(pulp.value(prob.objective),0))

print("---------All Completed for Network ")

Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
0:00:29  data pull from BQ
0:00:14  data processing, creating dictionaries
0:00:00  model set up
0:02:29  constraints set up
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               1800
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
Tried aggregator 2 times.
MIP Presolve eliminated 14739 rows and 186705 columns.
Aggregator did 80 substitutions.
Reduced MIP has 3051 rows, 67546 columns, and 131984 nonzeros.
Reduced MIP has 34220 binaries, 33326 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.75 sec. (366.15 ticks)
Found incumbent of value 2586754.030373 after 1.16 sec. (461.26 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 3051 rows, 67546 columns, and 131984 nonzeros.
Reduced MIP has 34220 binaries, 33326 generals, 0 SOSs, and 0 indic

In [5]:
# final_ded_details = pd.concat(ded_details_dict.values(), ignore_index=True)
ded_details.to_gbq('SRS_Matching.SRS_MATCHING_DED',
               'analytics-supplychain-thd',
               chunksize=None,
               if_exists='replace'
               )
# ow_df = pd.concat(ow_dict.values(), ignore_index=True)
one_way.to_gbq('SRS_Matching.SRS_MATCHING_OW',
               'analytics-supplychain-thd',
               chunksize=None,
               if_exists='replace'
               )

100%|██████████| 1/1 [00:00<?, ?it/s]
